In [8]:
from pymongo import MongoClient
from pprint import pprint
import json
from inventory import Inventory

with open('all_items.json') as f:
    allthings = json.load(f)

allcards = set(allthings['cards'])
allrelics = set(allthings['relics'])

client = MongoClient('mongodb://localhost:27017')
db = client.spire
status = db.command('serverStatus')
runs = db.runs

search = {
    'event.character_chosen': 'IRONCLAD', 
    'event.floor_reached': {
        '$gte': 40
    }, 
    'event.ascension_level': {
        '$gte': 15
    },
    "event.relics": {
        '$nin': ['PrismaticShard']
    },
    "event.is_beta": False,
    "event.is_endless": False,
    "event.relics": {"$in": ["Peace Pipe"]}, # todo: remove
    
}

num_runs = runs.count_documents(search)
print('num_runs', num_runs)

result = runs.aggregate([
    {'$match': search},
    {'$sample': { 'size': 1 } } # todo: remove
])

for r in result:
    floors = range(0, r['event']['floor_reached'])
    actions = {}
    run = r['event']
    
    # first step: reconstruct all actions performed on the player's inventory
    for floor in floors:
        actions[floor] = []

    # items_purchased
    for idx, floor in enumerate(run['item_purchase_floors']):
        item = run['items_purchased'][idx]
        if item in allcards:
            action = 'addCard'
        if item in allrelics:
            action = 'addRelic'
        actions[floor].append({
            'type': action,
            'name': item,
            'context': 'purchase',
        })

    # card_choices
    for choice in run['card_choices']:
        if choice['picked'] == 'SKIP':
            continue
        if choice['picked'] == 'Singing Bowl':
            continue
        actions[choice['floor']].append({
            'type': 'addCard',
            'name': choice['picked'],
            'context': 'pick',
        })

    # relics_obtained
    for e in run['relics_obtained']:
        actions[e['floor']].append({
            'type': 'addRelic',
            'name': e['key'],
            'context': 'relic_obtained',
        })

    # items_purged
    for idx, floor in enumerate(run['items_purged_floors']):
        item = run['items_purged'][idx]
        if item in allcards:
            action = 'removeCard'
        if item in allrelics:
            action = 'removeRelic'
        actions[floor].append({
            'type': action,
            'name': item,
            'context': 'purge',
        })

    # campfire_choices
    for choice in run['campfire_choices']:
        if choice['key'] == 'SMITH':
            action = {
                'type': 'upgradeCard',
                'name': choice['data'],
                'context': 'campfire',
            }
        elif choice['key'] == 'PURGE':
            action = {
                'type': 'removeCard',
                'name': choice['data'],
                'context': 'campfire',
            }
        actions[choice['floor']].append(action)

    # events
    for e in run['event_choices']:
        fe = actions[e['floor']]
        # when a card's tranformed, the old is in cards_transformed, the new is in cards_obtained
        if 'cards_transformed' in e:
            for item in e['cards_transformed']:
                fe.append({
                    'type': 'removeCard',
                    'name': item,
                    'context': 'event_transform_card',
                })
        if 'cards_removed' in e:
            for item in e['cards_removed']:
                fe.append({
                    'type': 'removeCard',
                    'name': item,
                    'context': 'event_remove_card',
                })
        if 'cards_upgraded' in e:
            for item in e['cards_upgraded']:
                fe.append({
                    'type': 'upgradeCard',
                    'name': item,
                    'context': 'event_upgrade_card',
                })
        if 'cards_obtained' in e:
            for item in e['cards_obtained']:
                fe.append({
                    'type': 'addCard',
                    'name': item,
                    'context': 'event_obtain_card',
                })
        if 'relics_lost' in e:
            for item in e['relics_lost']:
                fe.append({
                    'type': 'removeRelic',
                    'name': item,
                    'context': 'event_lost_relic',
                })
        if 'relics_obtained' in e:
            for item in e['relics_obtained']:
                fe.append({
                    'type': 'addRelic',
                    'name': item,
                    'context': 'event_obtain_relic',
                })

    # next step: replay all inventory changes floor-by-floor
    inventory_states = {}
    inv = Inventory('IRONCLAD')

    for floor in floors:
        if floor not in actions:
            continue

        # snapshot current state
        inventory_states[floor] = inv.export()

        # apply changes
        action_list = actions[floor]
        for action in action_list:
            if action['type'] == 'addCard':
                inv.add_card(action['name'])
            elif action['type'] == 'removeCard':
                inv.remove_card(action['name'])
            elif action['type'] == 'upgradeCard':
                inv.upgrade_card(action['name'])
            elif action['type'] == 'addRelic':
                inv.add_relic(action['name'])
            elif action['type'] == 'removeRelic':
                inv.remove_relic(action['name'])
            else:
                print("OOPS... action not found: ", action['type'])

    pprint(inventory_states)
    break


num_runs 636


ValueError: list.remove(x): x not in list